In [27]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
DFRAME_PATH="/content/drive/MyDrive/Colab Notebooks/aggregated_df.csv"

In [30]:
df = pd.read_csv(DFRAME_PATH)

In [31]:
df.head(10)

,author,book,text,text_no_punkt,lemmas,tags,tokens
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет. Материалы [к биографии]\nКак б...,Мы за пять лет Материалы к биографии Как будто...,пять год материал биография кроваво-красный ра...,4_NUMR 3_NOUN 9_NOUN 9_NOUN 15_ADJF 6_NOUN 8_V...,Мы за пять лет . Материалы [ к биографии ] Как...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...,Сидел он за большим письменным столом перед де...,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Да уж, — качал головой сдержанный Ре-ми. — Н...",Да уж качал головой сдержанный Ре-ми Нехорошо ...,качать голова сдержать ре-ми нехорошо нехорошо...,5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_ADVB 4_A...,"— Да уж , — качал головой сдержанный Ре-ми . —..."
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Поверит ли кто-нибудь, что нами за эти пять ле...",Поверит ли кто-нибудь что нами за эти пять лет...,поверить кто-нибудь пять год совместно м.г кор...,7_VERB 10_NPRO 4_NUMR 3_NOUN 9_ADVB 3_None 11_...,"Поверит ли кто-нибудь , что нами за эти пять л..."
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"8\nАверченко А. Избранные рассказы. М., 1985. ...",8 Аверченко А Избранные рассказы М 1985 С 7 9 ...,8 избранный рассказ м 1985 7 9 ежегодник 156 1...,1_None 9_ADJF 8_NOUN 1_NOUN 4_None 1_None 1_No...,"8 Аверченко А. Избранные рассказы . М. , 1985 ..."
5,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Помилуйте, такой праздник… Нужно, обязательн...",Помилуйте такой праздник Нужно обязательно поц...,помиловать праздник нужно обязательно поцелова...,9_VERB 8_NOUN 5_PRED 11_ADVB 12_INFN 7_NOUN 10...,"— Помилуйте , такой праздник… Нужно , обязател..."
6,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— Разбудить его в половине шестнадцатого.\n* *...,Разбудить его в половине шестнадцатого Закончи...,разбудить половина шестнадцатый закончить всё ...,9_INFN 8_NOUN 13_ADJF 8_GRND 3_PRCL 6_NOUN 7_N...,— Разбудить его в половине шестнадцатого . * *...
7,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Он встал, оделся, побрился и вышел на улицу.\n...",Он встал оделся побрился и вышел на улицу На у...,встать одеться побриться выйти улица улица сто...,5_VERB 6_VERB 8_VERB 5_VERB 5_NOUN 5_NOUN 11_V...,"Он встал , оделся , побрился и вышел на улицу ..."
8,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мозаика\nI\n— Я несчастный человек — вот что!\...,Мозаика I Я несчастный человек вот что Что за ...,мозаика i несчастный человек вздор поверить ув...,7_NOUN 1_None 10_ADJF 7_NOUN 5_NOUN 6_VERB 6_V...,Мозаика I — Я несчастный человек — вот что ! —...
9,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— Как записываешь?\n— В записной книжке. Хочеш...,Как записываешь В записной книжке Хочешь У мен...,записывать записной книжка хотеть минута откро...,11_VERB 8_ADJF 6_NOUN 6_VERB 6_NOUN 13_NOUN 6_...,— Как записываешь ? — В записной книжке . Хоче...


In [32]:
def get_document_vectorizer(frame, n_min=1, n=2, max_count=10000, column="text"):
    texts_vector = frame[column]
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    vectorizer.fit(texts_vector)
    return vectorizer

def get_author_vectorizer(frame, n_min=1, n=2, max_count=10000, column="text"):
    grouped_text = frame.groupby("author", as_index = False)[column].agg({column: ' '.join})
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    texts_vector = frame[column]
    vectorizer.fit(texts_vector)
    return vectorizer

In [33]:
def get_encoder(frame, column="author"):
    encoder = LabelEncoder()
    encoder.fit(frame[column])
    return encoder

In [34]:
label_encoder = get_encoder(frame=df)

In [35]:
y = label_encoder.transform(df["author"])
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [36]:
vectorizer = get_author_vectorizer(df_train, max_count=40000)
x_train = vectorizer.transform(df_train["text"])
x_test = vectorizer.transform(df_test["text"])

 CatBoost

In [13]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [37]:
from catboost import Pool, CatBoostClassifier

In [ ]:
По

In [38]:
df = df[["text"]] #


In [39]:
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [46]:
train_pool = Pool(
    df_train, 
    y_train, 
    text_features=["text"],
    
   
)
valid_pool = Pool(
    df_test, 
    y_test,
    text_features=["text"], 
   
)

catboost_params = {
    'iterations': 10000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 1000
}
model = CatBoostClassifier(**catboost_params)

In [47]:
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.7237258	test: 0.7472936	best: 0.7472936 (0)	total: 34.6ms	remaining: 5m 46s
1000:	learn: 0.9217411	test: 0.9499323	best: 0.9501579 (998)	total: 23.1s	remaining: 3m 27s
2000:	learn: 0.9394452	test: 0.9564727	best: 0.9564727 (1997)	total: 44.1s	remaining: 2m 56s
3000:	learn: 0.9525259	test: 0.9597429	best: 0.9597429 (2948)	total: 1m 4s	remaining: 2m 31s
4000:	learn: 0.9614344	test: 0.9609833	best: 0.9609833 (3997)	total: 1m 25s	remaining: 2m 8s
5000:	learn: 0.9711322	test: 0.9618854	best: 0.9621110 (4604)	total: 1m 51s	remaining: 1m 51s
6000:	learn: 0.9754172	test: 0.9629003	best: 0.9631258 (5840)	total: 2m 11s	remaining: 1m 27s
7000:	learn: 0.9821831	test: 0.9639152	best: 0.9640280 (6557)	total: 2m 31s	remaining: 1m 4s
8000:	learn: 0.9868065	test: 0.9645918	best: 0.9647046 (7921)	total: 2m 52s	remaining: 43s
9000:	learn: 0.9899639	test: 0.9651556	best: 0.9652684 (8966)	total: 3m 12s	remaining: 21.4s
9999:	learn: 0.9927830	test: 0.9651556	best: 0.9656067 (9621)	total: 3m 33s	